In [3]:
import googlemaps  
import pandas as pd
import numpy as np
import re
from time import sleep
import gmaps 
import getpass
from dotenv import load_dotenv

In [4]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

In [5]:
location = (-23.54158115282599, -46.63009071740232)  # center location
radius = 50000  # radius in meters
business = 'restaurant'
keyword = 'hamburger'

# requesting data
response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)

# appending our data to a list
results = []
results.extend(response['results'])

# if there is a next_page, get the data from it too (up to 60)
next_page = response['next_page_token']
while True:
    try:
        sleep(2)
        response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
        results.extend(response['results'])
        next_page = response['next_page_token']
    except:
        break

In [6]:
df = pd.DataFrame(results)
df.head()

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity
0,OPERATIONAL,"{'location': {'lat': -23.5838134, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,New Dog Hamburger,{'open_now': True},"[{'height': 642, 'html_attributions': ['<a hre...",ChIJWRNvGF9XzpQRetBgux1t6qw,"{'compound_code': 'C88G+FX São Paulo, State of...",3.0,4.3,ChIJWRNvGF9XzpQRetBgux1t6qw,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",9234,"R. Joaquim Floriano, 254 - Itaim Bibi, São Paulo"
1,OPERATIONAL,"{'location': {'lat': -23.5931406, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Mullequinho's Burguer,{'open_now': True},"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJvZnm7EVXzpQRrjB7C8kGepA,"{'compound_code': 'C848+PF São Paulo, State of...",1.0,4.3,ChIJvZnm7EVXzpQRrjB7C8kGepA,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",1593,"R. Min. Jesuíno Cardoso, 636 - Vila Olímpia, S..."
2,OPERATIONAL,"{'location': {'lat': -23.4278259, 'lng': -46.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Burger Boss,{'open_now': True},"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJY4qxzYSLzpQRsUt0L3vyLrw,"{'compound_code': 'HGC8+VR Aeroporto, Guarulho...",NaN,3.8,ChIJY4qxzYSLzpQRsUt0L3vyLrw,GOOGLE,"[restaurant, food, point_of_interest, establis...",255,"Rod. Hélio Smidt, s/n - Aeroporto, Guarulhos"
3,OPERATIONAL,"{'location': {'lat': -23.4632932, 'lng': -47.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Jeronimo Burger,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJr66kdhMRz5QRDCMlVFrVC3w,"{'compound_code': 'GXPC+M5 Araçariguama, State...",2.0,4.6,ChIJr66kdhMRz5QRDCMlVFrVC3w,GOOGLE,"[restaurant, food, point_of_interest, establis...",450,"Rod. Pres. Castello Branco, 2410, Araçariguama"
4,OPERATIONAL,"{'location': {'lat': -23.5634079, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,LE Burger Secreto,{'open_now': False},"[{'height': 2269, 'html_attributions': ['<a hr...",ChIJt1HwzNNZzpQRZeL9k1IJfVc,"{'compound_code': 'C8PM+JM Jardim Paulista, Sã...",2.0,4.3,ChIJt1HwzNNZzpQRZeL9k1IJfVc,GOOGLE,"[store, restaurant, food, point_of_interest, e...",4845,"R. Augusta, 2554 - Jardim Paulista, São Paulo"


In [7]:
df['url'] = 'https://www.google.com/maps/place/?q=place_id:'+df['place_id']
df = df[df['business_status'] == 'OPERATIONAL']

def return_lat(item):
    return item['location']['lat']

def return_lon(item):
    return item['location']['lng']

df['lat'] = df['geometry'].apply(return_lat)
df['lon'] = df['geometry'].apply(return_lon)
df = df.drop('geometry', axis='columns')

df = df.drop('icon_background_color', axis='columns')
df = df.drop('icon_mask_base_uri', axis='columns')
df = df.drop('opening_hours', axis='columns')

def get_neighborhood(item):
    try:
        result_str = item['compound_code']
        regex_result = re.findall(r'([.* ])(.+?),', result_str)
        neighb = regex_result[0][1]
    except:
        return np.nan
    else:
        return neighb
    
    
def get_city(item):
    try:
        result_str = item['compound_code']
        regex_result = re.findall(r'([.* ])(.+?),(.*)[-|,]', result_str)
        city = regex_result[0][2]
    except:
        return np.nan
    else:
        return city

df['neighborhood'] = df['plus_code'].apply(get_neighborhood)
df['city'] = df['plus_code'].apply(get_city)

df = df.drop('reference', axis='columns')
df = df.drop('scope', axis='columns')

df.head()    

,business_status,icon,name,photos,place_id,plus_code,price_level,rating,types,user_ratings_total,vicinity,url,lat,lon,neighborhood,city
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,New Dog Hamburger,"[{'height': 642, 'html_attributions': ['<a hre...",ChIJWRNvGF9XzpQRetBgux1t6qw,"{'compound_code': 'C88G+FX São Paulo, State of...",3.0,4.3,"[meal_takeaway, restaurant, food, point_of_int...",9234,"R. Joaquim Floriano, 254 - Itaim Bibi, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.583813,-46.672560,São Paulo,NaN
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Mullequinho's Burguer,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJvZnm7EVXzpQRrjB7C8kGepA,"{'compound_code': 'C848+PF São Paulo, State of...",1.0,4.3,"[meal_takeaway, restaurant, food, point_of_int...",1593,"R. Min. Jesuíno Cardoso, 636 - Vila Olímpia, S...",https://www.google.com/maps/place/?q=place_id:...,-23.593141,-46.683855,São Paulo,NaN
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Burger Boss,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJY4qxzYSLzpQRsUt0L3vyLrw,"{'compound_code': 'HGC8+VR Aeroporto, Guarulho...",NaN,3.8,"[restaurant, food, point_of_interest, establis...",255,"Rod. Hélio Smidt, s/n - Aeroporto, Guarulhos",https://www.google.com/maps/place/?q=place_id:...,-23.427826,-46.482943,Aeroporto,Guarulhos
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Jeronimo Burger,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJr66kdhMRz5QRDCMlVFrVC3w,"{'compound_code': 'GXPC+M5 Araçariguama, State...",2.0,4.6,"[restaurant, food, point_of_interest, establis...",450,"Rod. Pres. Castello Branco, 2410, Araçariguama",https://www.google.com/maps/place/?q=place_id:...,-23.463293,-47.029510,Araçariguama,NaN
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,LE Burger Secreto,"[{'height': 2269, 'html_attributions': ['<a hr...",ChIJt1HwzNNZzpQRZeL9k1IJfVc,"{'compound_code': 'C8PM+JM Jardim Paulista, Sã...",2.0,4.3,"[store, restaurant, food, point_of_interest, e...",4845,"R. Augusta, 2554 - Jardim Paulista, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.563408,-46.665866,Jardim Paulista,São Paulo


In [8]:
# Some lists to store the data we want
street = []
neighborhood = []
city = []
week_days = [[], [], [], [], [], [], []]
editorial_reviews = []
phones = []
total_reviews = []
websites = []
author_names = [[], [], [], [], []]
ratings = [[], [], [], [], []]
relative_times = [[], [], [], [], []]
texts = [[], [], [], [], []]
references = []

# string manipulation (useful for the current_opening_hours data section)
def replace_unicode(string):
    return string.replace('\u2009', ' ').replace('\u202f', ' ')

def decode_time(list_of_times):
    new_time = list(map(replace_unicode, list_of_times))
    return new_time

# For each place id, try to extract a particular information
for p_id in df['place_id']:
    place = map_client.place(p_id)
    street.append(place['result']['address_components'][1]['long_name'])
    neighborhood.append(place['result']['address_components'][2]['long_name'])
    city.append(place['result']['address_components'][3]['long_name'])
    try:
        hour_list = place['result']['current_opening_hours']['weekday_text']
    except:
        week_days[0].append(np.nan)
        week_days[1].append(np.nan)
        week_days[2].append(np.nan)
        week_days[3].append(np.nan)
        week_days[4].append(np.nan)
        week_days[5].append(np.nan)
        week_days[6].append(np.nan)
    else:
        hourly = decode_time(hour_list)
        idx = 0
        for c in hourly:
            pattern = '(\w*:)(.*)'
            regex = re.findall(pattern, c)
            week_days[idx].append(regex[0][1].strip())
            idx += 1
    
    try:
        editorial = place['result']['editorial_summary']['overview']
    except:
        editorial_reviews.append(np.nan)
    else:
        editorial_reviews.append(editorial)
    
    try:
        phone = place['result']['formatted_phone_number']
    except:
        phones.append(np.nan)
    else:
        phones.append(phone)
    try:
        t_r = place['result']['user_ratings_total']
    except:
        total_reviews.append(np.nan)
    else:
        total_reviews.append(t_r)
    
    try:
        website = place['result']['website']
    except:
        websites.append(np.nan)
    else:
        websites.append(website)
    
    # If the reviews section does not exist, append Nan
    try:
        reviews = place['result']['reviews']
    except:
        idx = 0
        while idx+1 < 5:
            author_names[idx].append(np.nan)
            ratings[idx].append(np.nan)
            relative_times[idx].append(np.nan)
            texts[idx].append(np.nan)
            idx += 1
    else:
        idx = 0
        for c in place['result']['reviews']:
            author_names[idx].append(c['author_name'])
            ratings[idx].append(c['rating'])
            relative_times[idx].append(c['relative_time_description'])
            texts[idx].append(c['text'])
            idx += 1

        while idx < 5:  # if there is less than 5 reviews, append nan to the remaining
            author_names[idx].append(np.nan)
            ratings[idx].append(np.nan)
            relative_times[idx].append(np.nan)
            texts[idx].append(np.nan)
            idx += 1
    
    references.append(place['result']['reference'])            
    

In [12]:
data = {'street':street, 'neighborhood':neighborhood, 'city':city, 
        'opening_hours_monday':week_days[0],'opening_hours_tuesday':week_days[1],'opening_hours_wednesday':week_days[2],'opening_hours_thursday':week_days[3],'opening_hours_friday':week_days[4],'opening_hours_saturday':week_days[5],'opening_hours_sunday':week_days[6],
        'editorial_review':editorial_reviews, 'phone':phones, 'website':websites, 
        'review_1':texts[0],'review_2':texts[1],'review_3':texts[2],'review_4':texts[3],'review_5':texts[4],
        'rating_1':ratings[0],'rating_2':ratings[1],'rating_3':ratings[2],'rating_4':ratings[3],'rating_5':ratings[4],
        'author_1':author_names[0],'author_2':author_names[1],'author_3':author_names[2],'author_4':author_names[3],'author_5':author_names[4],
        'relative_time_1':relative_times[0],'relative_time_2':relative_times[1],'relative_time_3':relative_times[2],'relative_time_4':relative_times[3],'relative_time_5':relative_times[4],'total_reviews':total_reviews,
        'reference':references}

In [14]:
place_search_df = pd.DataFrame(data=data)
place_search_df.head()

ValueError: All arrays must be of the same length

In [ ]:
final_df = df.set_index('place_id').join(place_search_df.set_index('reference'), lsuffix='_nearby', rsuffix='_place')
final_df.head()

In [ ]:
final_df.to_csv('final_df.csv', index=False)
final_df.columns

In [ ]:
locations = []
for c in range(len(df)):
    locations.append((final_df['lat'].iloc[c], final_df['lon'].iloc[c]))

In [ ]:
gmaps.configure(api_key='YOUR_API_KEY')

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(locations)

markers = gmaps.marker_layer([location])

fig.add_layer(heatmap_layer)
fig.add_layer(markers)
fig

In [ ]:
heatmap_layer.max_intensity = 2
heatmap_layer.point_radius = 10
heatmap_layer.dissipating  = True